In [1]:
import os
import json
import requests
import base64
from datetime import date
import re
import time
from pprint import pprint
from openpyxl import load_workbook
import xlsxwriter
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pyotp import TOTP
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from seleniumbase import Driver

In [2]:
brand = "Maxton Design"
print(f"Automated fulfillment started {brand}")

Automated fulfillment started Maxton Design


In [3]:
#Login Details

GORGIAS_URL = "https://mlperformance-1.gorgias.com"
GORGIAS_OD_USERNAME = 'amanina@mlperformance.co.uk'
GORGIAS_OD_API_KEY = 'a40765a261a5096b6c4f04d16b25bf655cb77edb6c397ee88b8267480caa54b8'

AUTHORIZATION = base64.b64encode(
    f"{GORGIAS_OD_USERNAME}:{GORGIAS_OD_API_KEY}".encode("utf-8")
).decode("utf-8")

HEADERS = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Basic {AUTHORIZATION}",
}

# print(HEADERS)

In [4]:
#Supplier details

supplier_list = {"Maxton Design": {
        "supplier_id" : 143818911, # supplier id in po shopify
        "supplier_email" : ["sales@maxtondesign.co.uk"],
        "customer_ids":[175694695], # cutomer id in gorgias
        "subject_pattern": "MAXTON DESIGN UK - Order \\d+ dispatched"
    }
}

In [5]:
def generate_ticket_list(tag_id):
    subject_pattern = supplier_list["Maxton Design"]["subject_pattern"]
    customer_ids = supplier_list["Maxton Design"]["customer_ids"]
    ticket_id_list = []
    for customer_id in customer_ids:
        # supplier_url = f"{GORGIAS_URL}/api/tickets?customer_id={customer_id}&order_by=created_datetime%3Adesc&page=1&per_page=30"
        supplier_url = f"{GORGIAS_URL}/api/tickets?customer_id={customer_id}"
        response = requests.get(supplier_url, headers=HEADERS).json()
        pprint(response)
        
        ticket_id_list.extend(
            [
                data["id"]
                for data in response["data"]
                if (
                    re.search(subject_pattern, data["subject"])
                    # and any(tag_id in all_tag.values() for all_tag in data["tags"])
                    # and data["status"] == "open"
                )
            ]
        )
    return ticket_id_list

ticket_id_list = generate_ticket_list('711016')
ticket_id_list

{'data': [{'assignee_team': None,
           'assignee_user': None,
           'channel': 'email',
           'closed_datetime': '2025-05-20T11:39:45.845164+00:00',
           'created_datetime': '2025-05-20T11:39:41+00:00',
           'custom_fields': {'31433': {'id': 31433,
                                       'value': 'Other::No Reply::Other'}},
           'customer': {'email': 'sales@maxtondesign.co.uk',
                        'firstname': 'Maxton',
                        'id': 175694695,
                        'lastname': 'Ltd',
                        'meta': {'name_set_via': 'email'},
                        'name': 'Maxton Design Ltd'},
           'excerpt': 'Payment Receipts - Auto Closed',
           'external_id': None,
           'from_agent': False,
           'id': 164752101,
           'integrations': [{'address': 'orders@mlperformance.co.uk',
                             'name': 'Ordering Department',
                             'type': 'gmail'}],
           'is_u

[161958675,
 161958672,
 160853678,
 160853677,
 159842207,
 159840256,
 159840251,
 159667860,
 157365659,
 156608546]

In [11]:
ticket_id_list = [161958675, 161958672]

In [12]:
# Fetch a single ticket from Gorgias
def get_single_ticket(ticket_id):
    url = f"{GORGIAS_URL}/api/tickets/{ticket_id}"
    response = requests.get(url, headers=HEADERS)
    
    if response.status_code != 200:
        print(f"Failed to fetch ticket {ticket_id}: {response.status_code}")
        print(f"Error message: {response.text}")
        return None
    
    try:
        response_json = response.json()
        print(f"Raw API response for ticket {ticket_id}:")
        #pprint(response_json)  # Print
        
        # Try different response structures
        ticket = response_json.get("data") or response_json
        if not ticket:
            print(f"Ticket {ticket_id}: No ticket data found in response.")
            return None
        
        print(f"Successfully fetched ticket {ticket_id}.")
        return ticket
    except ValueError as e:
        print(f"Ticket {ticket_id}: Failed to parse JSON response: {str(e)}")
        print(f"Raw response: {response.text}")
        return None

ticket_data = {}

for single_ticket_id in ticket_id_list:
    # Fetch the ticket
    ticket = get_single_ticket(single_ticket_id)
    ticket_data[single_ticket_id] = ticket
    if ticket:
        print(f"Processed ticket data for {single_ticket_id}:")
        # pprint(ticket)  # Print processed ticket data
    else:
        print("No ticket data to process.")

Raw API response for ticket 161958675:
Successfully fetched ticket 161958675.
Processed ticket data for 161958675:
Raw API response for ticket 161958672:
Successfully fetched ticket 161958672.
Processed ticket data for 161958672:


In [13]:
# Function to extract Reference Number 1 from ticket
def extract_reference_number(ticket, ticket_id):
    if not ticket:
        print(f"Ticket {ticket_id}: No data to process.")
        return None, ticket_id
    
    messages = ticket.get("messages", [])
    if not messages:
        print(f"Ticket {ticket_id}: No messages found.")
        return None, ticket_id
    
    body_text = messages[0].get("body_text", "")
    # Debug: Print body text excerpt
    body_excerpt = body_text[:200] + "..." if len(body_text) > 200 else body_text
    #print(f"Ticket {ticket_id} body excerpt:\n{body_excerpt}")
    
    match = re.search(r"Reference Number 1:\s*(\d+)", body_text)
    if match:
        reference_num = match.group(1)
        print(f"Ticket {ticket_id}: Reference Number 1: {reference_num}")
        return reference_num, ticket_id
    else:
        print(f"Ticket {ticket_id}: Reference Number 1 not found.")
        print(f"Full body text:\n{body_text}")
        return None, ticket_id

# Extract reference number
ticket_info = []

for single_ticket_id, ticket in ticket_data.items():
    if ticket:
        ref_num, ticket_id = extract_reference_number(ticket, single_ticket_id)
        ticket_link = f"{GORGIAS_URL}/app/ticket/{ticket_id}"
        if ref_num:
            ticket_info.append({"ticket_id": ticket_id, "reference_num": ref_num, "ticket_link": ticket_link})
            print(f"Stored: Ticket ID: {ticket_id}, Reference Number: {ref_num}, Link: {ticket_link}")
        else:
            print(f"Ticket ID: {ticket_id}, No reference number extracted, Link: {ticket_link}")
    else:
        print("Skipping extraction due to missing ticket data.")

Ticket 161958675: Reference Number 1 not found.
Full body text:
(https://fonts.gstatic.com)  (https://fonts.googleapis.com/css2?family=Lato:ital,wght@0,300;0,400;0,700;0,900;1,100&display=swap) Maxton Design Ltd   
   
  
  
   (https://www.maxtondesign.co.uk)  
  
   
   
  
  (https://www.maxtondesign.co.uk) www.maxtondesign.co.uk  
   
  
  
   
   
  
Your order is on its way! 
  
																                       and will be with you soon. 
   
  
  (https://www.maxtondesign.co.uk/account#/orders) view my orders  
   
   
  
Order ID: 251323   
 Track your delivery:  
 Tracking number: 7049511519 
 Tracking link: https://my.dxdelivery.com/ (https://my.dxdelivery.com/)  
    
 The comments for your order are:  
 Package shipped on 5/2/2025 via DX Secure Flex Next day. Tracking No 7049511519 
  
   
   
  
 
 
  * Your return may not be accepted or subject to a restocking fee if the protective film is tampered with, the fitting kit is missing, or it does not arrive back in the 